This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

# Introduction to deep learning for computer vision

## Introduction to convnets

**Instantiating a small convnet**

###### REDES CONVOLUCIONALES
    
    - Capas densas
        * aprenden patrones globales, en lugares especificos 
    - Capas Conv2D (filtros de 3x3 regularmente)
        * se observar pedacitos de imagen
        * dintingue patrones en cualquier lugar de la imagen
        * los patrones son invariantes ante traslaciones
        * hacen mapas de características (FM) un mapa de caracteristicas es una imagen transformada
        * son una especie de filtros que se fijan en diferentes aspectos y se guardan en mapas de características:
        * los mapas de características es un tensor de 3 dimensiones
        * stride: pasos del filtro convolucional
        * padding: agregar un renglon arriba, un renglón abajo, una columnna a la izquierda y una a la derecha, esto se hace para recuperar una imagen del mismo tamaño: no recomendado porque gasta muchos recursos y los resultados no mejoran demasiado 
    - Capas maxPooling2d (ventanas de 2x2 regularmente): valor maximo (promedio o mediana) dentro de un pool o recuadro
        * intenta resaltar los pixeles más importantes de la imagen
        * disminuye el mapa de caracteristicas (FM)
        * de una ventana o marco de datos, extrae el valor más alto (mediana, promedio) y a diferencia de las conv2D estás no se empalman
        * reduce la imagen a la mitad
        

Despues de cada de convolución no es necesario un max pooling

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
# definicion de tamaño de las imagenes
inputs = keras.Input(shape=(28, 28, 1))
# layer de convolución llamado x
#filtros en la capa (potencias de dos), 
#kernel_size = tamaño del filtro de convolucion 
# siempre se debe de poner el nombre de la capa anterior al final de 
# definir la capa 
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
# max pooling solo se define el tamaño de la ventana 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
# aplanamos la capa anterior
x = layers.Flatten()(x)
# capa densa de salida para la clasificacion
# puede ser más de una capa densa
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

**Displaying the model's summary**

In [2]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 128)         73856 

los params se obtienen derivados del filtro

capa 2: filtro (3x3) por el numero de capas anterior (1) + 1(este uno es de un error) x el numero de neuronas de la capa actual- ultima entrada)

**Training the convnet on MNIST images**

In [4]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255
model.compile(optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 70s 73ms/step - loss: 0.1545 - accuracy: 0.9517
Epoch 2/5
938/938 [==============================] - 68s 73ms/step - loss: 0.0444 - accuracy: 0.9867
Epoch 3/5
938/938 [==============================] - 68s 73ms/step - loss: 0.0306 - accuracy: 0.9908
Epoch 4/5
938/938 [==============================] - 69s 74ms/step - loss: 0.0232 - accuracy: 0.9929
Epoch 5/5
938/938 [==============================] - 29s 31ms/step - loss: 0.0179 - accuracy: 0.9943


**Evaluating the convnet**

evaluacion del modelo

In [5]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

313/313 [==============================] - 2s 6ms/step - loss: 0.0286 - accuracy: 0.9913
Test accuracy: 0.991


### The convolution operation

#### Understanding border effects and padding

#### Understanding convolution strides

### The max-pooling operation

**An incorrectly structured convnet missing its max-pooling layers**

In [6]:
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_no_max_pool = keras.Model(inputs=inputs, outputs=outputs)

In [7]:
model_no_max_pool.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_7 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 conv2d_8 (Conv2D)           (None, 22, 22, 128)       73856     
                                                                 
 flatten_2 (Flatten)         (None, 61952)             0         
                                                                 
 dense_2 (Dense)             (None, 10)                619530    
                                                                 
Total params: 712,202
Trainable params: 712,202
Non-trainab

## Training a convnet from scratch on a small dataset

### The relevance of deep learning for small-data problems

### Downloading the data

In [10]:
from google.colab import files
files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [11]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

La sintaxis del comando no es correcta.
"cp" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"chmod" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [ ]:
# liberando los archivos para poderlos leer

!kaggle competitions download -c dogs-vs-cats

In [ ]:
# carpetas de train

!unzip -qq train.zip

**Copying images to training, validation, and test directories**

In [ ]:
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

### Building the model

**Instantiating a small convnet for dogs vs. cats classification**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

**Configuring the model for training**

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

### Data preprocessing

**Using `image_dataset_from_directory` to read images**

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)

In [ ]:
import numpy as np
import tensorflow as tf
random_numbers = np.random.normal(size=(1000, 16))
dataset = tf.data.Dataset.from_tensor_slices(random_numbers)

In [ ]:
for i, element in enumerate(dataset):
    print(element.shape)
    if i >= 2:
        break

In [ ]:
batched_dataset = dataset.batch(32)
for i, element in enumerate(batched_dataset):
    print(element.shape)
    if i >= 2:
        break

In [ ]:
reshaped_dataset = dataset.map(lambda x: tf.reshape(x, (4, 4)))
for i, element in enumerate(reshaped_dataset):
    print(element.shape)
    if i >= 2:
        break

**Displaying the shapes of the data and labels yielded by the `Dataset`**

In [ ]:
for data_batch, labels_batch in train_dataset:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break

**Fitting the model using a `Dataset`**

un call back es una instruccion/funcion que nosotros queremos saber
sobre el proceso de entrenamiento (metrica o perdida) e ir guardando 
estos datos y decir a partir de que momento dejar de correr las 
epocas, es decir un callback nos podría ayudar a encontrar la
epoca minima a traves del loss

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint( #revisa la perdida
        filepath="convnet_from_scratch.keras",
        save_best_only=True, #guarda el mejor modelo
        monitor="val_loss") #monitoreal eval loss
]
# este callback guarda el modelo en el loss mas bajo sobre la 
# validación

history = model.fit(
    train_dataset,
    epochs=30,# se aumenta el numero de epocas para ver donde empieza
              # a sobreajustar el modelo
    validation_data=validation_dataset,
    callbacks=callbacks)

**Displaying curves of loss and accuracy during training**

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

**Evaluating the model on the test set**

In [ ]:
# se carga el modelo guardado de hace dos celdas 
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

### Using data augmentation
modificar las imagenes, rotar voltear, inclinar, hacerle zoom, etc

hacer esto es bueno, pero con medida, porque metemos imagenes repetidas

**Define a data augmentation stage to add to an image model**

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"), #voltear la foto
        layers.RandomRotation(0.1), #rotar la foto
        layers.RandomZoom(0.2), #ponerle zoom a la foto
    ]
)

**Displaying some randomly augmented training images**

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_dataset.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

**Defining a new convnet that includes image augmentation and dropout**

In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
# aquí se agrega el data augmentation unicamente sobre el train
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
# como hicimos el data augmantation hacemos drop-out
# es decir apagamos la mitad de las neuronas para evitar 
# sobreajuste
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

**Training the regularized convnet**

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch_with_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]

#el data augmantation necesita más epocas por eso las subimos a 100
history = model.fit(
    train_dataset,
    epochs=60, #orifinalmente eran 100
    validation_data=validation_dataset,
    callbacks=callbacks)

**Evaluating the model on the test set**

In [ ]:
# cuando se hace la evaluacion sobre el tests
# las imagenes con data aigmantation desaparecen para el test

test_model = keras.models.load_model(
    "convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

## Leveraging a pretrained model

### Feature extraction with a pretrained model

**Instantiating the VGG16 convolutional base**

aquí se decarga la arquitectura de la red

In [ ]:
conv_base = keras.applications.vgg16.VGG16( # en lugar de vgg16.VGG16 poner 'InceptionsV3' y jala también
    weights="imagenet", # que se quedue con los pesos del entrenamiento de 
                        # imagenet (conjunto de train) con el que se hizo la red   
                      
    include_top=False,  # que laparte de clasificación no me la dé
    input_shape=(180, 180, 3)) # tamaño de mis imagenes que metere en el modelo 

  # aquí se descarga el modelo 

In [ ]:
conv_base.summary()

#### Fast feature extraction without data augmentation

**Extracting the VGG16 features and corresponding labels**

In [ ]:
import numpy as np

def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels =  get_features_and_labels(train_dataset)
val_features, val_labels =  get_features_and_labels(validation_dataset)
test_features, test_labels =  get_features_and_labels(test_dataset)

In [ ]:
train_features.shape #mapa de caracteristicas

**Defining and training the densely connected classifier**

In [ ]:
# la linea anterior fue para ver qeu shape le dabamos a nuestra 
# red
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs) # se aplana el vector
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
      filepath="feature_extraction.keras",
      save_best_only=True,
      monitor="val_loss")
]
history = model.fit(
    train_features, train_labels,
    epochs=20,
    validation_data=(val_features, val_labels),
    callbacks=callbacks)

**Plotting the results**

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

Parece que hay sobreentrenamiento desde el principio

In [ ]:
test_model = keras.models.load_model(
    'feature_extraction.keras'
)

test_loss, test_acc = test_model.evaluate(test_features, test_labels)
print(f'test accuracy: {test_acc:.3f}')

#### Feature extraction together with data augmentation

**Instantiating and freezing the VGG16 convolutional base**

In [ ]:
conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)
conv_base.trainable = False # que el modelo vgg no sea reentrenable

**Printing the list of trainable weights before and after freezing**

In [ ]:
conv_base.trainable = True # que el modelo sea reentrenable
print("This is the number of trainable weights "
      "before freezing the conv base:", len(conv_base.trainable_weights))

In [ ]:
conv_base.trainable = False
print("This is the number of trainable weights "
      "after freezing the conv base:", len(conv_base.trainable_weights))

**Adding a data augmentation stage and a classifier to the convolutional base**

In [ ]:
# una vez construido nuestro modelo hacemos el data augmantation

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1), #rotacion
        layers.RandomZoom(0.2), # zoom
    ]
)

inputs = keras.Input(shape=(180, 180, 3)) #tamaño de imag de entyrada
x = data_augmentation(inputs) #data augmantation

#la siguiente linea cambia el tamañano de las imagenes a tamños con el cual se entrenó 
# originalmente el vgg es decir de 180 pasa a 255
x = keras.applications.vgg16.preprocess_input(x) # NUEVO preprocesamiento
x = conv_base(x) #modelo vgg
x = layers.Flatten()(x) #aplanamiento
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction_with_data_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset,
    callbacks=callbacks)

**Evaluating the model on the test set**

In [ ]:
test_model = keras.models.load_model(
    "feature_extraction_with_data_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

los resultados no mejoran mucho, pero al usar data augmantation evitamos el sobreajuste, es decir tarda en sobreajustar, lo cual es bueno y recomendable. extra: graficar los resultados

### Fine-tuning a pretrained model

lo que haremos ahora con nuestra red vgg, es reentrenar una parte de la red para mejorar el modelo

In [ ]:
conv_base.summary()

**Freezing all layers until the fourth from the last**

In [ ]:
conv_base.trainable = True # la base convolutiva es reentrenable
for layer in conv_base.layers[:-4]: # para las capas, excepto las ultimas cuatro
    layer.trainable = False # no son reentrenables

**Fine-tuning the model**

In [ ]:
# ya no es necesario volver a definir todo el modelo porque ya se corrio antes
# si no hubieramos declarado antes el modelo sí hay que poner toda
# la declaracion del modelo

model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="fine_tuning.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

In [ ]:
model = keras.models.load_model("fine_tuning.keras")
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

El modelo no mejora mucho, pero regularmente se mejora en 10% en promedio

## Summary